# Analisis de variables numericas parte 3

En este jupiter, buscamos comparar las medias, medianas y dispersiones de las variables numericas entre los clusteres de clientes. El objetivo es identificar diferencias estadisticamente significativas que nos permitan entender si los grupos de clientes tienen un comportamiento de compra distinto entre si. Para ello, aplicaremos pruebas como Welch, Mann-Whitney U y Levene.

El resultado de este analisis nos permitirá ir más allá de los datos y construir perfiles de clientes claros y accionables junto con lo que hemos ido recabando en anteriores notebooks. 

In [1]:
# Importamos las librerías necesarias
import pandas as pd
from scipy.stats import mannwhitneyu, levene, ttest_ind

df_customer = pd.read_csv("dataset/segmented_customers.csv")
df_customer.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,Cluster
0,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly,2
1,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly,1
2,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2
3,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly,2
4,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually,2


A continuación, crearemos dos funciones clave para automatizar nuestro análisis. La primera función, identify_numerical_cols, se encargará de identificar las columnas numéricas relevantes para nuestro estudio. Después, la función analyze_cluster_differences_custom_final ejecutará pruebas estadísticas para comparar esas variables entre clústeres.

El objetivo principal de estas pruebas es determinar si hay clústeres que son significativamente menores en términos de su media, mediana o desviación estándar en comparación con otros. Al usar pruebas como las de Welch y Mann-Whitney U para la media y mediana, y la de Levene para la desviación estándar, podremos validar nuestras hipótesis y obtener conclusiones sólidas sobre las características distintivas de cada clúster. Esto es fundamental para identificar los segmentos más pequeños y diferenciados de nuestros datos.

In [2]:
def identify_numerical_cols(df, cluster_col):
    """
    Identifica automáticamente las columnas numéricas de un DataFrame,
    excluyendo la columna de clúster.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.
        cluster_col (str): El nombre de la columna que contiene los clústeres.

    Returns:
        list: Una lista con los nombres de las columnas numéricas.
    """
    try:
        if cluster_col not in df.columns:
            raise KeyError(f"La columna '{cluster_col}' no se encontró en el DataFrame.")
        
        numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
        if cluster_col in numerical_cols:
            numerical_cols.remove(cluster_col)
        
        return numerical_cols
    except KeyError as e:
        print(f"Error: {e}")
        return []

In [3]:
def analyze_cluster_differences_custom_final(df, numerical_cols, cluster_col, target_cluster, alpha=0.05):
    """
    Realiza un análisis comparativo de un clúster específico contra todos los demás,
    identificando diferencias significativas en mediana, media y dispersión en la
    dirección de "menor que".

    Args:
        df (pd.DataFrame): El DataFrame con los datos.
        numerical_cols (list): La lista de columnas numéricas a analizar.
        cluster_col (str): El nombre de la columna de clústeres.
        target_cluster (int): El número del clúster a analizar.
        alpha (float): Nivel de significancia para las pruebas estadísticas.

    Returns:
        pd.DataFrame: Un DataFrame resumen con las diferencias significativas.
    """
    try:
        # Filtramos los datos del clúster de referencia
        df_target = df[df[cluster_col] == target_cluster]
        
        # Obtenemos la lista de clústeres a comparar (excluyendo el de referencia)
        other_clusters = [c for c in df[cluster_col].unique() if c != target_cluster]
        
        results = []
        
        # Iteramos sobre cada variable numérica
        for var in numerical_cols:
            median_less_clusters = []
            mean_less_clusters = []
            less_dispersed_clusters = []
            
            # Comparamos el clúster objetivo con cada uno de los otros clústeres
            for other_cluster in other_clusters:
                df_other = df[df[cluster_col] == other_cluster]
                
                # --- Test de Mediana (Mann-Whitney U) ---
                # Comparamos si la mediana del clúster de referencia es significativamente MENOR que la del otro.
                stat_mw, p_value_mw = mannwhitneyu(df_target[var], df_other[var], alternative='less')
                if p_value_mw < alpha:
                    median_less_clusters.append(str(other_cluster))
                
                # --- Test de Media (Welch's t-test) ---
                # Usamos la prueba de Welch (equal_var=False) para comparar medias cuando las varianzas son desiguales.
                # 'alternative='less'' comprueba si la media del clúster de referencia es significativamente MENOR.
                stat_tt, p_value_tt = ttest_ind(df_target[var], df_other[var], equal_var=False, alternative='less')
                if p_value_tt < alpha:
                    mean_less_clusters.append(str(other_cluster))

                # --- Test de Dispersión (Levene) ---
                # Identificamos clústeres significativamente MENOS dispersos.
                stat_levene, p_value_levene = levene(df_target[var], df_other[var])
                if p_value_levene < alpha and df_target[var].std() > df_other[var].std():
                    less_dispersed_clusters.append(str(other_cluster))
            
            # Construimos la fila para la tabla de resultados
            row = {
                'Variable': var,
                'Clúster de Referencia': target_cluster,
                'Clúster(es) con Mediana Significativamente Menor': ', '.join(median_less_clusters) if median_less_clusters else 'Ninguno',
                'Clúster(es) con Media Significativamente Menor (Welch)': ', '.join(mean_less_clusters) if mean_less_clusters else 'Ninguno',
                'Clúster(es) Significativamente Menos Dispersos': ', '.join(less_dispersed_clusters) if less_dispersed_clusters else 'Ninguno'
            }
            results.append(row)
            
        return pd.DataFrame(results)

    except Exception as e:
        print(f"Ocurrió un error durante el análisis: {e}")
        return pd.DataFrame() # Devuelve un DataFrame vacío en caso de error

In [5]:
# Identificamos las columnas numéricas del DataFrame
numerical_cols_identified = identify_numerical_cols(df_customer, 'Cluster')
print(f"Columnas numéricas identificadas para el análisis: {numerical_cols_identified}")

# Analizamos el Clúster _ y mostramos los resultados en la tabla
target_cluster_to_analyze = 5
summary_table = analyze_cluster_differences_custom_final(
    df=df_customer,
    numerical_cols=numerical_cols_identified,
    cluster_col='Cluster',
    target_cluster=target_cluster_to_analyze
)

# Mostramos la tabla resumen en formato Markdown para Jupyter Notebook
print(f"\n### Análisis Comparativo del Clúster {target_cluster_to_analyze} vs. Otros Clústeres\n")
print(summary_table.to_markdown(index=False))

Columnas numéricas identificadas para el análisis: ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases']

### Análisis Comparativo del Clúster 5 vs. Otros Clústeres

| Variable              |   Clúster de Referencia | Clúster(es) con Mediana Significativamente Menor   | Clúster(es) con Media Significativamente Menor (Welch)   | Clúster(es) Significativamente Menos Dispersos   |
|:----------------------|------------------------:|:---------------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------|
| Age                   |                       5 | 2, 3, 8, 4                                         | 2, 3, 8, 4                                               | Ninguno                                          |
| Purchase Amount (USD) |                       5 | Ninguno                                            | Ninguno                                                  | Ninguno